
# 🔍 Markov Full QC Notebook – Transition & Forward PD

Notebook này giúp bạn QC toàn bộ **Markov Engine**:

1. Kiểm tra **STATE SPACE** (BUCKETS, ABSORBING)
2. Kiểm tra **parent_fallback** và ma trận theo (Product × Score × MOB)
3. Kiểm tra **tỷ lệ fallback / dữ liệu yếu**
4. QC **Forward PD_12M** theo (STATE, MOB)
5. QC thêm **heatmap PD** và một vài phân tích.

> ⚠️ Lưu ý: bạn cần chỉnh lại đoạn **load dữ liệu** cho phù hợp project của bạn.


In [ ]:

# ============================================================
# 1) IMPORTS & SETUP
# ============================================================

import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Hiển thị đẹp
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

# Thêm project root vào sys.path nếu cần
root = Path(".").resolve()
if (root / "src").exists():
    sys.path.append(str(root))

from src.config import CFG, BUCKETS_CANON, ABSORBING_BASE
from src.rollrate.transition import compute_transition_by_mob, STATE_SPACE
from src.rollrate.pd_forward import (
    compute_forward_pd_one_record,
    compute_forward_pd,
    DEFAULT_STATE,
)



## 2️⃣ Load dữ liệu quá khứ (panel) để build ma trận

Tuỳ dự án, bạn có thể:

- Dùng `load_data()` nếu đã có sẵn hàm
- Hoặc đọc trực tiếp từ parquet / csv / database

Ở đây tôi để **placeholder**, bạn cần chỉnh lại cho đúng.


In [ ]:

# ============================================================
# 2) LOAD DATA – CHỈNH LẠI CHO PHÙ HỢP
# ============================================================

# Ví dụ 1: nếu bạn có hàm load_data trong src.data_loader
try:
    from src.data_loader import load_data
    df_raw = load_data()
    print(f"Loaded df_raw with shape = {df_raw.shape}")
except Exception as e:
    print("[WARN] Không load được dữ liệu bằng load_data().")
    print("      Hãy tự gán df_raw = <dataframe của bạn>.")
    df_raw = None

df_trans = df_raw  # nếu bạn có biến khác, gán lại ở đây



## 3️⃣ Build Transition Matrices – compute_transition_by_mob

Sử dụng engine đã có sẵn trong `src.rollrate.transition`:

```python
matrices_by_mob, parent_fallback = compute_transition_by_mob(df_trans)
```


In [ ]:

# ============================================================
# 3) BUILD TRANSITION MATRICES
# ============================================================

if df_trans is None:
    raise RuntimeError("df_trans đang = None. Hãy gán df_trans là panel dữ liệu lịch sử của bạn trước.")

matrices_by_mob, parent_fallback = compute_transition_by_mob(df_trans)
print(f"✅ Built matrices_by_mob for {len(matrices_by_mob)} products.")
print(f"✅ Parent fallback size = {len(parent_fallback)} (product, score) combinations.")



## 4️⃣ QC State Space (BUCKETS & ABSORBING)

Kiểm tra:

- `STATE_SPACE` dùng trong Markov
- `BUCKETS_CANON` từ config
- `ABSORBING_BASE` (PREPAY, WRITEOFF, ...)

Đảm bảo:

- Mọi state trong dữ liệu đều map vào STATE_SPACE
- Absorbing state đúng là hàng identity trong ma trận.


In [ ]:

# ============================================================
# 4) STATE SPACE QC
# ============================================================

print("STATE_SPACE:", STATE_SPACE)
print("BUCKETS_CANON:", BUCKETS_CANON)
print("ABSORBING_BASE:", ABSORBING_BASE)

# Kiểm tra xem absorbing state có nằm trong STATE_SPACE
missing_absorb = [s for s in ABSORBING_BASE if s not in STATE_SPACE]
if missing_absorb:
    print(f"⚠️ Absorbing state không nằm trong STATE_SPACE: {missing_absorb}")
else:
    print("✅ Tất cả absorbing state đều nằm trong STATE_SPACE.")



## 5️⃣ QC Parent Fallback (Product × Score)

- Liệt kê toàn bộ `(PRODUCT_TYPE, RISK_SCORE)` có parent fallback
- Kiểm tra:
  - Tổng hàng có = 1 không?
  - Xác suất default (DEFAULT_STATE) trung bình là bao nhiêu?


In [ ]:

# ============================================================
# 5) PARENT FALLBACK QC
# ============================================================

parent_summary = []

for (prod, score), P in parent_fallback.items():
    row_sum = P.sum(axis=1)
    default_prob = P.get(DEFAULT_STATE, pd.Series(0.0, index=P.index))

    parent_summary.append({
        "PRODUCT_TYPE": prod,
        "RISK_SCORE": score,
        "ROW_SUM_MIN": float(row_sum.min()),
        "ROW_SUM_MAX": float(row_sum.max()),
        f"P_to_{DEFAULT_STATE}_MEAN": float(default_prob.mean()),
        f"P_to_{DEFAULT_STATE}_MAX": float(default_prob.max()),
    })

df_parent_qc = pd.DataFrame(parent_summary).sort_values(
    ["PRODUCT_TYPE", "RISK_SCORE"]
).reset_index(drop=True)

print("📋 Parent fallback QC (top 20):")
display(df_parent_qc.head(20))



## 6️⃣ QC MOB-level – fallback, row-sum, default prob

Tạo bảng summary cho từng (Product, Score, MOB):

- `is_fallback`: có đang dùng parent fallback không?
- `ROW_SUM_MIN/ROW_SUM_MAX`: kiểm tra hàng có cộng về 1 không
- `P_to_DEFAULT_MEAN/MAX`: xác suất đi vào default_state


In [ ]:

# ============================================================
# 6) MOB-LEVEL QC
# ============================================================

mob_records = []

for prod, mob_dict in matrices_by_mob.items():
    for mob, score_dict in mob_dict.items():
        for score, obj in score_dict.items():
            # obj có thể là dict {"P": DataFrame, "is_fallback": bool, ...}
            if isinstance(obj, dict) and "P" in obj:
                P = obj["P"]
                is_fb = obj.get("is_fallback", False)
                reason = obj.get("reason", "")
            else:
                # Trường hợp cũ: obj chính là P
                P = obj
                is_fb = False
                reason = ""

            row_sum = P.sum(axis=1)
            default_prob = P.get(DEFAULT_STATE, pd.Series(0.0, index=P.index))

            mob_records.append({
                "PRODUCT_TYPE": prod,
                "RISK_SCORE": score,
                "MOB": int(mob),
                "IS_FALLBACK": bool(is_fb),
                "REASON": reason,
                "ROW_SUM_MIN": float(row_sum.min()),
                "ROW_SUM_MAX": float(row_sum.max()),
                f"P_to_{DEFAULT_STATE}_MEAN": float(default_prob.mean()),
                f"P_to_{DEFAULT_STATE}_MAX": float(default_prob.max()),
            })

df_mob_qc = pd.DataFrame(mob_records).sort_values(
    ["PRODUCT_TYPE", "RISK_SCORE", "MOB"]
).reset_index(drop=True)

print("📋 MOB-level QC (top 30):")
display(df_mob_qc.head(30))

print("\nTỷ lệ MOB bị fallback theo product:")
display(
    df_mob_qc.groupby("PRODUCT_TYPE")["IS_FALLBACK"]
    .mean()
    .rename("FALLBACK_RATIO")
    .reset_index()
)



## 7️⃣ Helper – Analyze PD_12M theo (STATE, MOB) cho 1 segment

Hàm dưới đây sẽ tạo **bảng PD_12M** cho toàn bộ STATE × MOB trong 1 segment
`(product, score)`.

Dùng để check:

- MOB thấp (0–4) có PD hợp lý không?
- PD có tăng dần theo MOB không?
- State xấu (DPD60+, DPD90+) có PD cao hơn state tốt (DPD0) không?


In [ ]:

# ============================================================
# 7) HELPER – PD GRID theo (STATE, MOB)
# ============================================================

def analyze_pd_by_state_and_mob(
    matrices_by_mob,
    parent_fallback,
    product: str,
    score: str,
    mob_min: int = 0,
    mob_max: int = 12,
    horizon: int = 12,
    default_state: str = DEFAULT_STATE,
) -> pd.DataFrame:
    """
    Trả về DataFrame:
        index  = MOB
        columns= STATE
        value  = PD_12M
    """
    # Lấy danh sách state chuẩn từ STATE_SPACE / BUCKETS_CANON
    if STATE_SPACE:
        states = list(STATE_SPACE)
    else:
        from src.config import ABSORBING_BASE
        states = list(dict.fromkeys(list(BUCKETS_CANON) + list(ABSORBING_BASE)))

    records = []

    for mob in range(mob_min, mob_max + 1):
        for st in states:
            try:
                pd_12m, _, _ = compute_forward_pd_one_record(
                    current_state=st,
                    current_mob=mob,
                    product=product,
                    score=score,
                    matrices_by_mob=matrices_by_mob,
                    parent_fallback=parent_fallback,
                    horizon=horizon,
                    default_state=default_state,
                    debug=False,
                )
            except Exception as e:
                print(f"[WARN] Lỗi compute_forward_pd_one_record(state={st}, mob={mob}): {e}")
                pd_12m = 0.0

            records.append({
                "MOB": mob,
                "STATE": st,
                "PD_12M": float(pd_12m),
            })

    df = pd.DataFrame(records)
    df_pivot = df.pivot_table(index="MOB", columns="STATE", values="PD_12M")
    return df_pivot



## 8️⃣ Chạy thử PD_12M grid cho 1 segment (Product × Score)

Ở đây tôi chọn **product & score đầu tiên** trong `matrices_by_mob` để minh hoạ.
Bạn nên thay bằng segment mà bạn quan tâm (vì thường PD bất thường chỉ xuất hiện ở 1 vài segment).

In [ ]:

# ============================================================
# 8) SAMPLE – PD GRID CHO 1 SEGMENT
# ============================================================

# Lấy product, score đầu tiên từ matrices_by_mob
sample_prod = None
sample_score = None

if matrices_by_mob:
    sample_prod = next(iter(matrices_by_mob.keys()))
    # lấy MOB bất kỳ để xem danh sách score
    any_mob = next(iter(matrices_by_mob[sample_prod].keys()))
    sample_score = next(iter(matrices_by_mob[sample_prod][any_mob].keys()))

print("Sample segment:", sample_prod, sample_score)

if sample_prod is not None and sample_score is not None:
    pd_grid = analyze_pd_by_state_and_mob(
        matrices_by_mob=matrices_by_mob,
        parent_fallback=parent_fallback,
        product=str(sample_prod),
        score=str(sample_score),
        mob_min=0,
        mob_max=12,
        horizon=12,
    )
    display(pd_grid)
else:
    print("⚠️ Không tìm được segment mẫu từ matrices_by_mob.")



## 9️⃣ Vẽ heatmap PD_12M theo (MOB, STATE)

Giúp bạn nhìn nhanh:

- State tốt / xấu phân bổ như thế nào theo MOB
- PD có tăng dần theo MOB không.


In [ ]:

# ============================================================
# 9) HEATMAP PD_12M (MOB × STATE)
# ============================================================

if 'pd_grid' in locals():
    plt.figure(figsize=(10, 6))
    plt.imshow(pd_grid.values, aspect="auto", origin="lower")
    plt.colorbar(label="PD_12M")
    plt.yticks(ticks=range(len(pd_grid.index)), labels=pd_grid.index)
    plt.xticks(ticks=range(len(pd_grid.columns)), labels=pd_grid.columns, rotation=45, ha="right")
    plt.title(f"Heatmap PD_12M – Product={sample_prod}, Score={sample_score}")
    plt.xlabel("STATE")
    plt.ylabel("MOB")
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Chưa có pd_grid để vẽ heatmap.")



## 🔟 (Optional) Backtest Markov vs Actual Default

Phần này phụ thuộc khá nhiều vào cấu trúc dữ liệu actual của bạn.
Gợi ý:

- Dùng `pipeline_lifecycle.py` để build **lifecycle actual + forecast**
- Hoặc dùng `forecast_full_history` + `add_del_metrics_simple` để tạo **DEL90_PCT model**
- So sánh với **DEL90_PCT actual** từ dữ liệu lịch sử.

Để tránh notebook quá gắn chặt với 1 cấu trúc dữ liệu cụ thể, tôi để phần này dạng **khung** để bạn hoàn thiện thêm nếu muốn.


In [ ]:

# ============================================================
# 10) GỢI Ý BACKTEST (KHUNG) – tuỳ biến theo dữ liệu thực tế
# ============================================================

# Ví dụ: nếu bạn đã có module forecast_full_history & add_del_metrics_simple
try:
    from src.rollrate.forecast_full_history import forecast_full_history
    from src.rollrate.del_metrics_simple import add_del_metrics_simple

    # 1) Full-history forecast từ MOB0 -> max_mob
    fc_full = forecast_full_history(
        df_raw=df_trans,
        matrices_by_mob=matrices_by_mob,
        max_mob=36,
    )

    print("✅ forecast_full_history shape:", fc_full.shape)

    # 2) Thêm DEL metrics đơn giản
    df_fc_del = add_del_metrics_simple(fc_full)
    print("✅ df_fc_del shape:", df_fc_del.shape)
    display(df_fc_del.head())

    print("""
    Gợi ý tiếp theo:
    - Từ df_fc_del: lấy DEL90_PCT model tại một horizon (ví dụ MOB=12)
    - Từ dữ liệu actual: build DEL90_PCT thực tế tại MOB=12 (lifecycle actual)
    - So sánh mean, median, distribution để đánh giá model.
    """)
except Exception as e:
    print("[INFO] Không import được forecast_full_history/add_del_metrics_simple hoặc df_trans chưa sẵn sàng.")
    print("      Bạn có thể tự hoàn thiện phần backtest theo cấu trúc dữ liệu của bạn.")
    print("Exception:", e)
